# Banking Feature Store Demo

This notebook demonstrates how to use Feast for banking feature management with:
- Offline feature retrieval for model training
- Online feature retrieval for real-time inference
- Feature services for organized feature access
- Simple on-demand feature transformations


In [ ]:
# Import required libraries
import pandas as pd
from feast import FeatureStore
from datetime import datetime, timedelta

print("🚀 Banking Feature Store Demo")
print("=" * 50)


## 1. Initialize Feature Store


In [ ]:
# Initialize the feature store
fs = FeatureStore(repo_path="feature_repo")

print("✅ Feature store initialized")


## 2. Apply and Materialize Features

In [ ]:
# Apply feature definitions to registry
print("🔄 Applying feature definitions...")
! feast -c feature_repo apply
print("✅ Features applied successfully")

# Materialize features to online store
print("🔄 Materializing features to online store...")
fs.materialize_incremental(end_date=datetime.now())
print("✅ Features materialized successfully")


## 3. List Available Feature Services


In [ ]:
# List all available feature services
feature_services = fs.list_feature_services()

print("📋 Available Feature Services:")
print("=" * 40)
for service in feature_services:
    print(f"🎯 {service.name}")
    print(f"   Description: {service.description}")
    print(f"   Features: {len(service.features)} feature views")
    print()


## 4. Offline Feature Retrieval (for Model Training)


In [ ]:
# Create entity dataframe for offline retrieval
print("📊 OFFLINE FEATURE RETRIEVAL (Model Training)")
print("=" * 50)

# Sample customer IDs
sample_customers = ["CUST_000001", "CUST_000002", "CUST_000003"]
recent_timestamp = pd.Timestamp('2025-09-15')

# Create entity dataframe
entity_df = pd.DataFrame({
    'customer_id': sample_customers,
    'event_timestamp': [recent_timestamp] * len(sample_customers)
})

print(f"📅 Using timestamp: {recent_timestamp}")
print(f"👥 Sample customers: {sample_customers}")

# Retrieve features using Customer Charter Feature Service
print("\n🎯 Using 'customer_charter_service' for offline retrieval...")
try:
    training_df = fs.get_historical_features(
        entity_df=entity_df,
        features=fs.get_feature_service("customer_charter_service"),
        full_feature_names=True
    ).to_df()
    
    print(f"✅ Retrieved {len(training_df)} rows with {len(training_df.columns)} features")
    print("\n📊 Sample data:")
    print(training_df.head())
    
except Exception as e:
    print(f"❌ Error: {e}")


## 5. Online Feature Retrieval (for Real-time Inference)


In [ ]:
# Online feature retrieval for real-time inference
print("⚡ ONLINE FEATURE RETRIEVAL (Real-time Inference)")
print("=" * 50)

# Use a customer with recent data
customer_id = "CUST_000465"
print(f"🎯 Getting features for customer: {customer_id}")

# Retrieve features using Customer Charter Feature Service
print("\n🎯 Using 'customer_charter_service' for online retrieval...")
try:
    online_features = fs.get_online_features(
        features=fs.get_feature_service("customer_charter_service"),
        entity_rows=[{"customer_id": customer_id}],
        full_feature_names=True
    ).to_dict()
    
    print(f"✅ Retrieved {len(online_features)} features")
    print("\n📊 Feature values:")
    for feature_name, value in online_features.items():
        if feature_name != 'customer_id':
            print(f"   {feature_name}: {value[0]}")
    
except Exception as e:
    print(f"❌ Error: {e}")


## 6. Multiple Feature Services Demo


In [ ]:
# Demonstrate different feature services
print("🎯 MULTIPLE FEATURE SERVICES DEMO")
print("=" * 40)

services_to_demo = [
    "customer_charter_service",
    "call_prediction_service",
    "fraud_detection_service"
]

for service_name in services_to_demo:
    print(f"\n🎯 Using '{service_name}'...")
    try:
        features = fs.get_online_features(
            features=fs.get_feature_service(service_name),
            entity_rows=[{"customer_id": customer_id}],
            full_feature_names=True
        ).to_dict()
        
        print(f"   ✅ Retrieved {len(features)} features")
        # Show first few features
        feature_count = 0
        for feature_name, value in features.items():
            if feature_name != 'customer_id' and feature_count < 3:
                print(f"   {feature_name}: {value[0]}")
                feature_count += 1
        if len(features) > 4:
            print(f"   ... and {len(features) - 4} more features")
            
    except Exception as e:
        print(f"   ❌ Error: {e}")


## 7. Simple On-Demand Feature Transformation


In [ ]:
# Demonstrate the actual Feast on-demand feature view
print("🔄 ON-DEMAND FEATURE VIEW DEMO")
print("=" * 40)

# Get features using the on-demand risk service
print("🎯 Using 'ondemand_risk_service' with on-demand transformations...")
try:
    ondemand_features = fs.get_online_features(
        features=fs.get_feature_service("ondemand_risk_service"),
        entity_rows=[{"customer_id": customer_id}],
        full_feature_names=True
    ).to_dict()
    
    print(f"✅ Retrieved {len(ondemand_features)} features")
    print("\n📊 On-demand feature values:")
    for feature_name, value in ondemand_features.items():
        if feature_name != 'customer_id':
            print(f"   {feature_name}: {value[0]}")
    
    # Show the calculated risk score
    if 'calculate_risk_score__risk_score' in ondemand_features:
        risk_score = ondemand_features['calculate_risk_score__risk_score'][0]
        print(f"\n🎯 On-Demand Risk Score: {risk_score:.1f}/100")
        print(f"   Risk Level: {'LOW' if risk_score > 70 else 'MEDIUM' if risk_score > 40 else 'HIGH'}")
    
except Exception as e:
    print(f"❌ Error: {e}")

print("\n💡 This demonstrates a real Feast on-demand feature view!")
print("   The risk score is calculated in real-time during feature retrieval.")


## 8. Summary


In [ ]:
# Updated Summary with On-Demand Features
print("📊 BANKING FEATURE STORE DEMO SUMMARY")
print("=" * 50)
print("✅ Feature store initialized and configured")
print("✅ Features applied and materialized")
print("✅ Offline feature retrieval for model training")
print("✅ Online feature retrieval for real-time inference")
print("✅ Feature services for organized feature access")
print("✅ On-demand feature view with real-time transformations")
print("\n🎯 Key Benefits:")
print("   • Centralized feature management")
print("   • Consistent feature access across teams")
print("   • Real-time and batch feature serving")
print("   • Feature versioning and lineage")
print("   • Easy feature discovery and reuse")
print("   • Real-time feature transformations")
print("\n🏦 Perfect for banking use cases:")
print("   • Customer risk assessment")
print("   • Fraud detection")
print("   • Call prediction")
print("   • Transaction analysis")
print("   • Customer segmentation")


In [ ]:
print("📊 BANKING FEATURE STORE DEMO SUMMARY")
print("=" * 50)
print("✅ Feature store initialized and configured")
print("✅ Features applied and materialized")
print("✅ Offline feature retrieval for model training")
print("✅ Online feature retrieval for real-time inference")
print("✅ Feature services for organized feature access")
print("✅ On-demand transformations demonstrated")
print("\n🎯 Key Benefits:")
print("   • Centralized feature management")
print("   • Consistent feature access across teams")
print("   • Real-time and batch feature serving")
print("   • Feature versioning and lineage")
print("   • Easy feature discovery and reuse")
print("\n🏦 Perfect for banking use cases:")
print("   • Customer risk assessment")
print("   • Fraud detection")
print("   • Call prediction")
print("   • Transaction analysis")
print("   • Customer segmentation")
